# 모델 배포 및 추론



## 1. 환경 구성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

python path: /home/ec2-user/SageMaker/Kor-LLM-On-SageMaker/5-Lab05-Summarization is added
sys.path:  ['/home/ec2-user/SageMaker/Kor-LLM-On-SageMaker/5-Lab05-Summarization/notebooks', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python310.zip', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/lib-dynload', '', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages', '/home/ec2-user/SageMaker/Kor-LLM-On-SageMaker/5-Lab05-Summarization']


In [2]:
%store -r model_s3_path
print("model_s3_path: ", model_s3_path)


model_s3_path:  {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-06-30-05-52-1-2024-06-30-05-52-18-207/output/model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


## 2. 추론 이미지 가져오기



In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/SageMaker/.xdg/config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon
sagemaker bucket: sagemaker-us-east-1-057716757052
sagemaker session region: us-east-1


In [4]:
import sagemaker
sagemaker.__version__

'2.224.2'

In [5]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  session=sess,
  version="2.0.2",
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.0.2-gpu-py310-cu121-ubuntu22.04


## 3. SageMaker Model 생성

In [6]:
instance_type = "ml.p4d.24xlarge"
num_GPUSs = 8

In [7]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config

health_check_timeout = 1200 # 20 minutes

import time
sm_endpoint_name = "llama3-endpoint-{}".format(int(time.time()))
print("sm_endpoint_name: \n", sm_endpoint_name)

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model",       # Path to the model in the container
  'SM_NUM_GPUS': f"{num_GPUSs}",        # Number of GPU used per replica
  'MAX_INPUT_LENGTH': "8000",           # Max length of input text
  'MAX_TOTAL_TOKENS': "8096",           # Max length of the generation (including input text)
  'MAX_BATCH_PREFILL_TOKENS': "16182",  # Limits the number of tokens that can be processed in parallel during the generation
  'MESSAGES_API_ENABLED': "true",       # Enable the OpenAI Messages API
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  # path to s3 bucket with model, we are not using a compressed model
  # {'S3DataSource':{'S3Uri': "s3://...",'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  model_data=model_s3_path,
  image_uri=llm_image,
  env=config
)

sm_endpoint_name: 
 llama3-endpoint-1719727500


/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4. 모델 배포

In [8]:
# Deploy model to an endpoint
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  endpoint_name=sm_endpoint_name,
  container_startup_health_check_timeout=health_check_timeout, # 20 minutes to give SageMaker the time to download and merge model
)

------------!

## 5. 모델 추론

### Helper 함수

In [9]:
from scripts.inference_util import (
    print_json,
    create_messages_parameters,
)


  


### SageMaker Predictor 로 추론

In [31]:

# system_prompt = "You are a helpful assistant."
# user_prompt = "Tell me something about Amazon SageMaker?"
# messages, parameters = create_messages_parameters(system_prompt = system_prompt, user_prompt = user_prompt)   

In [32]:
# chat = llm.predict({"messages" :messages, **parameters})
# print_json(chat)
# print(chat["choices"][0]["message"]["content"].strip())
# print_json(chat["choices"][0]["message"]["content"].strip())



### Boto3 InvokeEndpoint() 호출

In [12]:
import json

from scripts.inference_util import (
    create_boto3_request_body,
    invoke_endpoint_sagemaker,
    print_ww
)

In [33]:
# system_prompt = "You are a helpful assistant and write only in English"
# user_prompt = "How to make cake?"
# request_body = create_boto3_request_body(system_prompt=system_prompt, user_prompt=user_prompt)
# request_body

In [35]:
# import time
# s = time.perf_counter()

# # sm_endpoint_name = "llama3-endpoint-mnist-1719625657"
# response = invoke_endpoint_sagemaker(endpoint_name = sm_endpoint_name, 
#                          pay_load = request_body)    

# elapsed_async = time.perf_counter() - s

# print(f"elapsed time: {round(elapsed_async,3)} second")
# parsed_data = json.loads(response)
# print(json.dumps(parsed_data, indent=4, ensure_ascii=False))
# answer = parsed_data["choices"][0]["message"]["content"].strip()
# print_json(answer)
# print_ww(answer)


## 요약 추론

In [36]:
%store -r full_test_data_json

In [64]:

from scripts.inference_util import (
    get_message_from_dataset,
    extract_system_user_prompt,
    run_inference,
    generate_response
)

In [66]:
messages, full_test_dataset, rand_idx = get_message_from_dataset(
                                        sample_dataset_json_file = full_test_data_json, verbose=False)    
generate_response(messages, sm_endpoint_name, full_test_dataset, rand_idx)    

elapsed time: 1.029 second
**Query:**
{'messages': [{'role': 'system', 'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.'}, {'role': 'user', 'content': 'Please summarize the goals for journalist in this text:\n\n한국 건설사 수주 기대감 높아 성공시 제2 중동 붐 전망까지 삼성물산·현대건설 컨소시엄 10억달러 규모 터널공사 수주 이미지출처 연합뉴스 사우디아라비아의 실세 무함마드 빈살만 제1왕위 계승자 왕세자 가 사우디를 개방적이고 온건한 이슬람 국가로 바꾸겠다고 선언했다. 국제유가 상승으로 사우디 아라비아의 국부가 늘면서 대규모 인프라 투자로 이어지자 한국 건설사들의 수주 기대감이 높아지고 있다. 대표적인 것이 네옴시티 로 사우디 실권자인 무함마드 빈 살만 왕세자가 주도하는 이 프로젝트는 총 사업비만 무려 5000억달러 약 640조원 에 달하는 세계 최대 규모 인프라 사업이다. 이미 물밑에서 수주 경쟁이 치열한 이 사업은 한국 건설사들이 대규모 수주에 성공할 경우 제 2의 중동 붐 을 일으킬 것이란 전망까지 나온다. 1일 증권업계 및 건설업계에 의하면 사우디발 인프라 건설 훈풍으로 국내 건설사들의 수주 기대감이 커지고 있다. 사우디는 2016년 발표한 비전 2030 에 의해 국가전략을 수행하고 있는데 이는 석유에 의존해온 경제를 첨단 제조업 중심으로 전환하기 위한 프로젝트로 네옴시티도 이 중 하나다. 이미 사우디는 비공개로 입찰을 진행 국내에선 삼성

# 엔드포인트 삭제

In [67]:
# llm.delete_model()
# llm.delete_endpoint()